# Backpacking Dashboard

Integrate Photos (with timestamps and GPS coordinates) with a GPS route

In [4]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import flask
import geopandas
import glob
import ipywidgets as widgets
import json
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import scipy


from dash.dependencies import Input, Output  # Load Data
from datetime import datetime
from GPSPhoto import gpsphoto
from jupyter_dash import JupyterDash
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler

In [5]:
#https://stackoverflow.com/questions/40452759/pandas-latitude-longitude-to-distance-between-successive-rows
#vectorized haversine function, for determining the distance between two locations on earth


def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [6]:
#https://stackoverflow.com/questions/63787612/plotly-automatic-zooming-for-mapbox-maps
#Based off a collection of coordinates, finds optimal zoom and centering for a plotly mapbox


def zoom_center(lons: tuple=None, lats: tuple=None, lonlats: tuple=None,
                format: str='lonlat', projection: str='mercator',
                width_to_height: float=2.0) -> (float, dict):

    """Finds optimal zoom and centering for a plotly mapbox.
    Must be passed (lons & lats) or lonlats.
    Temporary solution awaiting official implementation, see:
    https://github.com/plotly/plotly.js/issues/3434
    
    Parameters
    --------
    lons: tuple, optional, longitude component of each location
    lats: tuple, optional, latitude component of each location
    lonlats: tuple, optional, gps locations
    format: str, specifying the order of longitud and latitude dimensions,
        expected values: 'lonlat' or 'latlon', only used if passed lonlats
    projection: str, only accepting 'mercator' at the moment,
        raises `NotImplementedError` if other is passed
    width_to_height: float, expected ratio of final graph's with to height,
        used to select the constrained axis.
    
    Returns
    --------
    zoom: float, from 1 to 20
    center: dict, gps position with 'lon' and 'lat' keys

    >>> print(zoom_center((-109.031387, -103.385460),
    ...     (25.587101, 31.784620)))
    (5.75, {'lon': -106.208423, 'lat': 28.685861})
    """
    if lons is None and lats is None:
        if isinstance(lonlats, tuple):
            lons, lats = zip(*lonlats)
        else:
            raise ValueError(
                'Must pass lons & lats or lonlats'
            )
    
    maxlon, minlon = max(lons), min(lons)
    maxlat, minlat = max(lats), min(lats)
    center = {
        'lon': round((maxlon + minlon) / 2, 6),
        'lat': round((maxlat + minlat) / 2, 6)
    }
    
    # longitudinal range by zoom level (20 to 1)
    # in degrees, if centered at equator
    lon_zoom_range = np.array([
        0.0007, 0.0014, 0.003, 0.006, 0.012, 0.024, 0.048, 0.096,
        0.192, 0.3712, 0.768, 1.536, 3.072, 6.144, 11.8784, 23.7568,
        47.5136, 98.304, 190.0544, 360.0
    ])
    
    if projection == 'mercator':
        margin = 1.2
        height = (maxlat - minlat) * margin * width_to_height
        width = (maxlon - minlon) * margin
        lon_zoom = np.interp(width, lon_zoom_range, range(20, 0, -1))
        lat_zoom = np.interp(height, lon_zoom_range, range(20, 0, -1))
        zoom = round(min(lon_zoom, lat_zoom), 2)
    else:
        raise NotImplementedError(
            f'{projection} projection is not implemented'
        )
    
    return zoom, center

In [7]:
#manually upload photos
#btn_upload = widgets.FileUpload()
#btn_upload

In [8]:
#btn_upload.value

In [9]:
#enter path to your folder of images
path_imagefolder = 


In [10]:
#get data from images (coordinates and timepoints) and image file names
image_list = os.listdir(path_imagefolder)
image_list = [a for a in image_list if a.endswith('jpg')]
data_list = []
ID_list = []
for i, a in enumerate(image_list):
    data = gpsphoto.getGPSData(path_imagefolder + f'\\{a}')
    data_list.append(data)
    ID_list.append(a)
    
#create dataframe
df = pd.DataFrame(data_list)
df['ID_list'] = ID_list

#convert altitude meters of feet
df['Altitude_feet'] = df['Altitude'] * 3.28084

#DETELE THIS PART?
#df['Date'].isna().sum()
only_na = df[df['Date'].isna()]

#drop NA columns
df.dropna(inplace=True)

#reset index for NA
df.reset_index(drop=True, inplace=True)

In [11]:
df_events = []

for event in range(len(df)):
    date_event = datetime.strptime(df['Date'][event], "%m/%d/%Y")
    time_event = datetime.strptime(df['UTC-Time'][event], "%H:%M:%S")
    combine_event = datetime.combine(datetime.date(date_event), datetime.time(time_event))
    df_events.append(combine_event)

    
#create data column of timepoints
df['Events'] = df_events

#sort dataframe by timepoints
df = df.sort_values(by='Events')
df.reset_index(inplace=True)

#create data columns of selected and not selected for dashboard
df['Zeros'] = 0
df['Selected'] = 0

#normalize timestamps
scaler = MinMaxScaler()
df['Events_normalized'] = scaler.fit_transform(df['Events'].values.reshape(-1,1))

#difference between each timestamp
df['Events_difference'] = df.Events_normalized - df.Events_normalized.shift()
df['Events_difference'][0] = 0
df['ID_order'] = np.arange(len(df))



C:\Users\peter\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,Latitude,Longitude,Altitude,UTC-Time,Date,ID_list,Altitude_feet,Events,Zeros,Selected,Events_normalized,Events_difference,ID_order
0,0,40.936522,-122.873121,2128.540,23:1:11,07/26/2020,20200726_160115.jpg,6983.399174,2020-07-26 23:01:11,0,0,0.000000,0.000000,0
1,1,40.936522,-122.873121,2128.540,23:1:11,07/26/2020,20200726_160204.jpg,6983.399174,2020-07-26 23:01:11,0,0,0.000000,0.000000,1
2,2,40.936545,-122.873089,2109.086,23:1:50,07/26/2020,20200726_160214.jpg,6919.573712,2020-07-26 23:01:50,0,0,0.000079,0.000079,2
3,3,40.938664,-122.876137,2129.788,23:46:32,07/26/2020,20200726_164634.jpg,6987.493662,2020-07-26 23:46:32,0,0,0.005527,0.005448,3
4,4,40.938664,-122.876137,2129.788,23:46:32,07/26/2020,20200726_164936.jpg,6987.493662,2020-07-26 23:46:32,0,0,0.005527,0.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,96,40.941007,-122.889681,2288.250,13:30:18,08/01/2020,20200801_063021.jpg,7507.382130,2020-08-01 13:30:18,0,0,0.983421,0.000995,97
98,98,40.939504,-122.881042,2259.520,13:42:29,08/01/2020,20200801_064233.jpg,7413.123597,2020-08-01 13:42:29,0,0,0.984906,0.001485,98
99,99,40.939504,-122.881042,2259.520,13:42:29,08/01/2020,20200801_064253.jpg,7413.123597,2020-08-01 13:42:29,0,0,0.984906,0.000000,99
100,100,40.939124,-122.880017,2309.300,13:47:52,08/01/2020,20200801_064815.jpg,7576.443812,2020-08-01 13:47:52,0,0,0.985562,0.000656,100


In [12]:
#convert events to universial time??
df['Events'] = df['Events'].dt.tz_localize('UTC')

In [13]:
#Convert to local time
df['Events_local'] = df['Events'].dt.tz_convert('US/Pacific')

In [14]:
#generate static image for display on website
image_directory = path_imagefolder
list_of_images = df['ID_list'].tolist()

static_image_route = '/static/'

In [15]:
#go get the GPS track, probably move this up to the beginning with getting the other data

df_gps = geopandas.read_file(r"C:\Users\peter\Desktop\datascience career\trinity_alps_2020.geojson")

combined = []

for a in range(len(df_gps.geometry[0])):
    coords=list(df_gps.geometry[0][a].coords)
    combined += coords
    
df_geo = pd.DataFrame(combined, columns =['Longitude', 'Latitude', 'Altitude']) 
df_geo['Altitude_feet'] = df_geo['Altitude'] * 3.28084

In [16]:
# find distance between each coordinate on route
df_geo['Distance'] = haversine(df_geo.Latitude.shift(), df_geo.Longitude.shift(),
                 df_geo.Latitude, df_geo.Longitude)

#convert km to miles and its good
#add cumulative distance

#convert to miles
df_geo['Distance_miles'] = df_geo['Distance'] * 0.621371

#calculate cumulative distance
df_geo['Cumulative_distance_miles']= df_geo['Distance_miles'].cumsum()

#create data column of the index of data points
df_geo['Route_order'] = np.arange(len(df_geo))

#enter in zero for initial coordinate, I assume because it can't subtract from anything
df_geo.Distance[0] = 0
df_geo.Distance_miles[0] = 0
df_geo.Cumulative_distance_miles[0] = 0

#create a normalized distance
df_geo['Cumulative_distance_normalized'] = df_geo.Cumulative_distance_miles / df_geo.Distance_miles.sum()


C:\Users\peter\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\peter\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\peter\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
#find zoom and center for map
zoom, center = zoom_center(
    lons= list(df_geo.Longitude),
    lats= list(df_geo.Latitude),
    width_to_height = 4.0
)



10.77
{'lon': -122.894272, 'lat': 40.971777}


In [18]:
#https://stackoverflow.com/questions/47534715/get-nearest-point-from-each-other-in-pandas-dataframe

#find the closest point to another in a dataframe



mat = scipy.spatial.distance.cdist(df_geo[['Latitude','Longitude']], 
                              df[['Latitude','Longitude']], metric='euclidean')


In [20]:
#create data frame with index as each route coordinate and columns as picture coordinates and each value being the distance 
#between the the two
new_df = pd.DataFrame(mat, index=df_geo['Route_order'], columns=df['ID_order']) 
#new_df

ID_order,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
Route_order,,,,,,,,,,,,,,,,,,,,,
0,0.061025,0.061025,0.060999,0.064444,0.064444,0.065018,0.069656,0.069656,0.071583,0.071770,...,0.087552,0.087406,0.087384,0.087084,0.078983,0.078165,0.069410,0.069410,0.068325,0.015402
1,0.060674,0.060674,0.060647,0.064093,0.064093,0.064666,0.069305,0.069305,0.071232,0.071419,...,0.087201,0.087055,0.087033,0.086733,0.078631,0.077814,0.069059,0.069059,0.067974,0.015051
2,0.060322,0.060322,0.060296,0.063742,0.063742,0.064315,0.068953,0.068953,0.070880,0.071067,...,0.086849,0.086703,0.086681,0.086381,0.078280,0.077462,0.068707,0.068707,0.067623,0.014699
3,0.059971,0.059971,0.059945,0.063390,0.063390,0.063964,0.068602,0.068602,0.070529,0.070716,...,0.086498,0.086352,0.086330,0.086030,0.077929,0.077111,0.068356,0.068356,0.067271,0.014348
4,0.059619,0.059619,0.059593,0.063039,0.063039,0.063613,0.068251,0.068251,0.070178,0.070365,...,0.086146,0.086000,0.085979,0.085679,0.077577,0.076759,0.068005,0.068005,0.066920,0.013997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3019,0.059678,0.059678,0.059651,0.063096,0.063096,0.063670,0.068308,0.068308,0.070235,0.070422,...,0.086205,0.086058,0.086037,0.085736,0.077635,0.076817,0.068062,0.068062,0.066977,0.014055
3020,0.060030,0.060030,0.060003,0.063448,0.063448,0.064021,0.068660,0.068660,0.070587,0.070774,...,0.086557,0.086410,0.086389,0.086088,0.077986,0.077169,0.068414,0.068414,0.067329,0.014407
3021,0.060382,0.060382,0.060355,0.063800,0.063800,0.064373,0.069012,0.069012,0.070938,0.071125,...,0.086909,0.086762,0.086740,0.086440,0.078338,0.077521,0.068766,0.068766,0.067681,0.014759


In [21]:
#create a dictionary list of the dataframe columns, find ten cloest
dict = {}


# creating a list of dataframe columns 
columns = list(new_df) 
  
for i in columns: 
    test = new_df.sort_values(by=[i])
    dict[i] =test[i].index[0:10]
  
    #find 10 closest
    


In [22]:
#transpose it
df_close = pd.DataFrame(dict)
df_close = df_close.T

,0,1,2,3,4,5,6,7,8,9
0,181,180,2872,182,2873,179,178,2871,2874,2875
1,181,180,2872,182,2873,179,178,2871,2874,2875
2,181,180,2873,2872,182,179,178,2874,2875,177
3,2863,191,2862,190,2864,192,189,2861,2865,193
4,2863,191,2862,190,2864,192,189,2861,2865,193
...,...,...,...,...,...,...,...,...,...,...
97,2814,2813,2812,233,2823,234,2818,232,2822,2816
98,2849,2848,205,206,207,2850,204,2847,208,203
99,2849,2848,205,206,207,2850,204,2847,208,203
100,2852,2851,202,2853,201,2850,203,200,2854,204


In [23]:
# NEED TO CHANE THIS SO THAT THE SELECTED VALUE IS NOT THE CLOSEST VALUE BUT THE LOWEST? 

In [24]:

#df_geo.iloc[df_close.iloc[0]].Cumulative_distance_normalized

181     0.082913
180     0.082439
2872    0.917047
182     0.083387
2873    0.917521
179     0.081965
178     0.081965
2871    0.916573
2874    0.917966
2875    0.917966
Name: Cumulative_distance_normalized, dtype: float64

In [25]:
previous_location = df_geo.iloc[df_close.iloc[0]].min().Cumulative_distance_normalized
#previous_location

0.08196483748512415

In [26]:
# creating a list of dataframe columns 
distance_difference = []
best_index = []
best_spacetime = []
best_cumulative_time = []

previous_location = df_geo.iloc[df_close.iloc[0].min()].Cumulative_distance_normalized

# creating a list of dataframe columns 
i = range(len(df_close))
  
for i in columns: 
    
    distance_difference.append( (df_geo.iloc[df_close.iloc[i]].Cumulative_distance_normalized - previous_location ))
    #print(distance_difference[i])
    temporal_difference = df['Events_difference'][i]
    #print(temporal_difference)
    spacetime = (temporal_difference - (distance_difference[i]))
    #print(spacetime)
    df_spacetime = pd.DataFrame(spacetime)
    #print(df_spacetime)
    
    best_index.append (   int(df_spacetime.abs().idxmin()) )
    #print(best_index)
    best_spacetime.append ( float(df_spacetime.abs().min() ))
    #print(best_spacetime)
    
    best_cumulative_time.append(df_geo['Cumulative_distance_miles'][best_index[i]])
    
    #best_index.append(spacetime.index[min(spacetime, key=abs)])
    #best_spacetime.append( min(spacetime, key=abs) )
                 
    previous_location = df_geo.iloc[best_index[i]].Cumulative_distance_normalized
    #print(previous_location)
    
   
    
    

In [27]:
df['Cumulative_distance_miles'] = best_cumulative_time


,index,Latitude,Longitude,Altitude,UTC-Time,Date,ID_list,Altitude_feet,Events,Zeros,Selected,Events_normalized,Events_difference,ID_order,Events_local,Cumulative_distance_miles
0,0,40.936522,-122.873121,2128.540,23:1:11,07/26/2020,20200726_160115.jpg,6983.399174,2020-07-26 23:01:11+00:00,0,0,0.000000,0.000000,0,2020-07-26 16:01:11-07:00,3.223720
1,1,40.936522,-122.873121,2128.540,23:1:11,07/26/2020,20200726_160204.jpg,6983.399174,2020-07-26 23:01:11+00:00,0,0,0.000000,0.000000,1,2020-07-26 16:01:11-07:00,3.223720
2,2,40.936545,-122.873089,2109.086,23:1:50,07/26/2020,20200726_160214.jpg,6919.573712,2020-07-26 23:01:50+00:00,0,0,0.000079,0.000079,2,2020-07-26 16:01:50-07:00,3.223720
3,3,40.938664,-122.876137,2129.788,23:46:32,07/26/2020,20200726_164634.jpg,6987.493662,2020-07-26 23:46:32+00:00,0,0,0.005527,0.005448,3,2020-07-26 16:46:32-07:00,3.428773
4,4,40.938664,-122.876137,2129.788,23:46:32,07/26/2020,20200726_164936.jpg,6987.493662,2020-07-26 23:46:32+00:00,0,0,0.005527,0.000000,4,2020-07-26 16:46:32-07:00,3.428773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,96,40.941007,-122.889681,2288.250,13:30:18,08/01/2020,20200801_063021.jpg,7507.382130,2020-08-01 13:30:18+00:00,0,0,0.983421,0.000995,97,2020-08-01 06:30:18-07:00,35.078757
98,98,40.939504,-122.881042,2259.520,13:42:29,08/01/2020,20200801_064233.jpg,7413.123597,2020-08-01 13:42:29+00:00,0,0,0.984906,0.001485,98,2020-08-01 06:42:29-07:00,35.601927
99,99,40.939504,-122.881042,2259.520,13:42:29,08/01/2020,20200801_064253.jpg,7413.123597,2020-08-01 13:42:29+00:00,0,0,0.984906,0.000000,99,2020-08-01 06:42:29-07:00,35.601927
100,100,40.939124,-122.880017,2309.300,13:47:52,08/01/2020,20200801_064815.jpg,7576.443812,2020-08-01 13:47:52+00:00,0,0,0.985562,0.000656,100,2020-08-01 06:47:52-07:00,35.657851


In [28]:
#for each column, find 5 lowest distance values. Then find the 5 corresponding cumulative distances
#for each column, find the its increamental increase in normalized time
#for each column subtract the current cumulative distance from the previous one. To get distance increase.
#I guess for the first timepoint the distance increase is just from 0, as in the time increase. 
#Compare distance increase to time increase, then pick accordingly 

In [29]:
test = df['Events_local']

def daylight_times(test):
    start = (min(test)).round(freq = 'D') - pd.Timedelta(days=2.25)
    end = (max(test)).round(freq = 'D') + pd.Timedelta(days=2.25)
    
    timepoints_18 = pd.date_range(start=start, end= (end + pd.Timedelta(days=0.5)), freq= '24H')
    timepoints_6 = pd.date_range(start= (start + pd.Timedelta(days=0.5)) , end=end, freq= '24H')
    
    return timepoints_18,timepoints_6

timepoints_18, timepoints_6 = daylight_times(test)


### Generate Dashboard

In [32]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = [dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


# ------------------------------------------------------------------------------
# App layout


info = dbc.Card(
    [
        dbc.FormGroup(
            [
                dcc.Graph(
                    id='my_topo',
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dcc.Graph(
                    id='my_route'
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dcc.Graph(
                    id='my_timeline'
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dcc.Slider(
                    id='my-slider',
                    min=0,
                    max=(len(list_of_images) - 1),
                    step=1,
   #                 value=0,
                ),
            ]
        ),
    ],
    body=True, style={'backgroundColor': '#323130'}
)


app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    
                    
                    
                        dbc.Row(
                            html.Img(id='image' , style={'height' : '675px', 'padding-top' : 20, 'padding-left' : 20}) ,justify="center"),
                    
                    
                md=7), 
                dbc.Col(info, md=5),
            ],
            align="start", justify="center"
        ),
    ],
    fluid=True,style={'backgroundColor':'#323130'}
)


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components


# Add a static image route that serves images from desktop
# Be *very* careful here - you don't want to serve arbitrary files
# from your computer or server
@app.server.route('{}<image_path>.jpg'.format(static_image_route))
def serve_image(image_path):
    image_name = '{}.jpg'.format(image_path)
    if image_name not in list_of_images:
        raise Exception('"{}" is excluded from the allowed static files'.format(image_path))
    return flask.send_from_directory(image_directory, image_name)



@app.callback(
    [dash.dependencies.Output('image', 'src'),
     dash.dependencies.Output('my_topo', 'figure'),
     dash.dependencies.Output('my_timeline', 'figure'),
     dash.dependencies.Output('my_route', 'figure')],
    [dash.dependencies.Input('my_topo', 'clickData'),
     dash.dependencies.Input('my_route', 'clickData'),
     dash.dependencies.Input('my_timeline', 'clickData')])

def update_image_src_map_timeline_route(clickData_map, clickData_route, clickData_timeline ):
#    return static_image_route + list_of_images[value], now just at end



    ctx = dash.callback_context
    
    if not ctx.triggered:
        button_id = 0
    else:
        
        if ctx.triggered[0]['prop_id'] == 'my-slider.value':
            button_id = button_id = ctx.triggered[0]['value']
            
        else:
            button_id = ctx.triggered[0]['value']['points'][0]['customdata'][0]
        
    
    
    print(button_id)
    value = button_id
    

#make map
    dff = df.copy()
    dff['Selected'] = dff['Selected']+0.1
    dff.iloc[value, dff.columns.get_loc('Selected')] = 1
    
    
    fig_map = px.scatter_mapbox(df_geo, lat="Latitude", lon="Longitude", opacity=0.5, zoom=zoom, center=center, color="Altitude_feet", hover_data=["Cumulative_distance_miles", "Cumulative_distance_normalized", "Route_order"])
    
    fig2_map = px.scatter_mapbox(dff, lat="Latitude", color_discrete_sequence=['white'], lon="Longitude",hover_name="Events", hover_data=["Altitude_feet", "ID_list", "ID_order"], zoom=zoom,  opacity = 0.7, size = "Selected", custom_data=["ID_order"])
    fig_map.add_trace(fig2_map.data[0])
    
    
    
    
    
    fig_map.update_layout(mapbox_style= "white-bg",mapbox_layers=[
            {
                "below": 'traces',
                "sourcetype": "raster",
                "source": [
                    "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
                ]
            }
        ]
    )
    fig_map.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
    fig_map.update_layout(coloraxis_showscale=True)
    
    fig_map.update_layout(coloraxis_colorbar_len=0.5)
    fig_map.update_layout(coloraxis_colorbar_yanchor = 'bottom')
    fig_map.update_layout(coloraxis_colorbar_bgcolor = 'rgba(0, 0, 0, 0)') 
    fig_map.update_layout(coloraxis_colorbar_xanchor = 'right')
    fig_map.update_layout(coloraxis_colorbar_xpad = 5)
    fig_map.update_layout(coloraxis_colorbar_ypad = 5)
    fig_map.update_layout(coloraxis_colorbar_x = 1.0)
    fig_map.update_layout(coloraxis_colorbar_y = 0.5)
    fig_map.update_layout(coloraxis_colorbar_tickfont_color = 'white')
    fig_map.update_layout(coloraxis_colorbar_title_font_color = 'white')
    
    
#make timeline
    fig_timeline = px.scatter(dff, x='Events_local', y = 'Zeros', color_discrete_sequence=['white'], opacity = 0.7, size = "Selected", height = 200 ,custom_data=["ID_order"])
    fig_timeline.update_layout(xaxis_showgrid=True, yaxis_showgrid=True)
    fig_timeline.update_layout(coloraxis_showscale=False)
    
    fig_timeline.update_yaxes(visible=False, showticklabels=True)
    fig_timeline.update_yaxes(range=[0, 0])
    fig_timeline.update_xaxes(title=None, visible=True, showticklabels=True,ticks="inside")
    
    for i in range(len(timepoints_6)):
        fig_timeline.add_vrect(x0= timepoints_18[i] ,
                  x1= timepoints_6[i],
                  fillcolor= 'black',
                  opacity=0.4,
                  line_width=0)
    

    fig_timeline.update_xaxes(range= [ (min(dff['Events_local'])- pd.Timedelta(days=0.25)) , (max(dff['Events_local'])+ pd.Timedelta(days=0.25))   ])
    
    fig_timeline.update_layout(height=100, margin={"r":0,"t":0,"l":0,"b":0} ,plot_bgcolor='#323930', paper_bgcolor='#323130', font_color="white" )
    
 #   fig_timeline.update_layout(clickmode='event+select')
    
#make route

    dff_geo = df_geo.copy()
    #dff_geo['Selected'] = dff_geo['Selected']+0.1
    #dff_geo.iloc[value, dff_geo.columns.get_loc('Selected')] = 1
    

    fig_route = go.Figure(data=go.Scatter(x=dff_geo.Cumulative_distance_miles, y=dff_geo.Altitude_feet, mode='markers',opacity = 0.9, marker_color= dff_geo.Altitude_feet))
    
    fig2_route = px.scatter(dff, x="Cumulative_distance_miles", y="Altitude_feet", color_discrete_sequence=['white'], opacity = 0.7, hover_name="Events", hover_data=["Altitude_feet", "ID_list", "ID_order"],
                          size = "Selected", custom_data=["ID_order"])
    
    
    fig_route.add_trace(fig2_route.data[0])
    
    
    fig_route.update_layout(xaxis_showgrid=True, yaxis_showgrid=True)
    fig_route.update_layout(coloraxis_showscale=False)
    
    fig_route.update_xaxes(title=None, visible=True, showticklabels=True,ticks="inside")
    
    
    
    fig_route.update_yaxes(visible=False, showticklabels=True)
    #fig.update_yaxes(range=[0, 0])
    #fig.update_xaxes(title=None, visible=True, showticklabels=True)
    
    fig_route.update_layout(height=150, margin={"r":0,"t":0,"l":0,"b":0} ,plot_bgcolor='#323930', paper_bgcolor='#323130', font_color="white")
    
    fig_route.update_traces(showlegend=False)
    
    fig_route.update_geos(fitbounds="locations") 
    
    
    
    
    return static_image_route + list_of_images[value], fig_map, fig_timeline, fig_route

#mode='jupyterlab'   inline   external

app.run_server(mode='inline')

0
